###### -----------------START--------------------------------------------

In [1]:
import json

In [2]:
import os

In [3]:
from tqdm import tqdm

In [4]:
train_file_path = '/home/aritra/cric/train_questions.json'
val_file_path = '/home/aritra/cric/val_questions.json'
test_file_path = '/home/aritra/cric/test_v1_questions.json'

In [5]:
# Training Set

with open(train_file_path, "r") as file:
     train_json = json.load(file)

In [6]:
# Validation Set

with open(val_file_path, "r") as file:
     val_json = json.load(file)

In [7]:
# Test Set

with open(test_file_path, "r") as file:
     test_json = json.load(file)

In [8]:
len(train_json)

365235

In [9]:
len(val_json)

43112

In [10]:
len(test_json)

86003

In [11]:
train_json[1099]['question']

'which brown animal walking in the field could be used for transporting people'

In [12]:
val_json[1099]['question']

'is there an object that is a type of public transports'

In [13]:
test_json[1099]['question']

'can the ceramic bird spread wings'

### ------------------------------Extracting Data of Training Set-------------------------------------------------------------------------------



In [14]:
questionList = []
answerList = []
imgList = []

In [15]:
train_json[2]['image_id']

1005

#### iter 1: from 0 , 149000 -> error1.txt -> 159
#### iter 2: from 150000 , 240000 -> error2.txt -> 34
#### iter 3: from 240000 , 365235 ->error3.txt -> 121

In [16]:
# verifying
indexToExclude = []

with open('error1.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExclude.append(number)
        
with open('error2.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExclude.append(number)
        
with open('error3.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExclude.append(number)

In [17]:
len(indexToExclude)

314

In [18]:
for i in tqdm(range(len(train_json))):
    
    if i in indexToExclude:
        continue
        
    pointer = train_json[i]
    
    questionList.append(pointer['question'])
    answerList.append(pointer['answer'])
    imgList.append(pointer['image_id'])

100%|█████| 365235/365235 [00:01<00:00, 321343.12it/s]


In [19]:
len(questionList), len(answerList), len(imgList)

(364921, 364921, 364921)

In [20]:
len(list(set(answerList)))

1442

### ---------------------------------------Map Creation--------------------------------------------------------

In [21]:
def findUnique(targetList):
    
    uniqueList = []
    
    for word in targetList:
        if word not in uniqueList:
            uniqueList.append(word)
    
    return uniqueList

In [22]:
len(findUnique(answerList))

1442

In [23]:
# creating word to number mapping

mapping = {}
counter = 0

uniqueAnsList = findUnique(answerList)

for word in uniqueAnsList:
    
    if word not in mapping:
        
        mapping[word] = counter
        counter += 1

In [24]:
uniqueAnsList[0:5]

['no', 'small', 'picture', 'table', 'bookshelf']

In [25]:
numOfClasses = max(mapping.values())
numOfClasses

1441

In [26]:
len(mapping)

1442

In [27]:
# creating number to word mapping

reverse_mapping = dict([(value, key) for key, value in mapping.items()])

### --------------------------------------Processing of Training Set--------------------------------------------------------------------

In [ ]:
labels = []

for i in range(len(answerList)):
    labels.append( mapping[ answerList[i] ] )

In [ ]:
len(labels)

In [ ]:
scores = []

for i in tqdm(range(len(answerList))):
    
    s = [0] * (numOfClasses+1)
    s[ mapping[ answerList[i]] ] = 1
    
    scores.append(s)

In [ ]:
len(scores)

In [ ]:
imgPathList = []
filepath = '/home/aritra/cric/images/img/'

for i in tqdm(range(len(imgList))):
    
    imgName = str(imgList[i]) + '.jpg'
    concatedPath = os.path.join(filepath,imgName)
    
    imgPathList.append(concatedPath)

In [ ]:
from datasets import load_dataset
from datasets import Dataset
import datasets
from PIL import Image
import torch

In [ ]:
imgPathList[0:5]

In [ ]:
len(imgPathList)

In [ ]:
listToDictionary = {'questions':questionList, 'labels': labels, 'scores': scores, 'images':imgPathList}
modified_train_set = Dataset.from_dict(listToDictionary)

In [ ]:
# mapping each filepath to images in the directory

modified_train_set = modified_train_set.cast_column("images", datasets.Image())

In [ ]:
modified_train_set

### ------------------------------------------------Extracting Validation Set---------------------------------------------

In [ ]:
questionList_val = []
answerList_val = []
imgList_val = []

In [ ]:
# collecting the index containing errorneous images

indexToExcludeVal = []
with open('error_validation.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExcludeVal.append(number)

with open('error_validation2.txt', 'r') as file:
    for line in file:
        number = int(line.strip())  # Convert the read line to an integer
        indexToExcludeVal.append(number)


In [ ]:
# excluding the index containing errorneous images

for i in tqdm(range(len(val_json))):
    
    if (i in indexToExcludeVal):
        continue
        
    pointer = val_json[i]
    
    questionList_val.append(pointer['question'])
    answerList_val.append(pointer['answer'])
    imgList_val.append(pointer['image_id'])

43112 -> 43068 -> 33175

In [ ]:
len(questionList_val), len(answerList_val), len(imgList_val)

In [ ]:
uniqueAnswerListVal = list(set(answerList_val))
len(uniqueAnswerListVal)

In [ ]:
# check if all the uniques answers are present in the mapping

y,n = 0,0
store = []
for i in range(len(answerList_val)):
    
    word = answerList_val[i]
    
    if word in mapping:
        y += 1
    else:
        n+=1
        store.append(i)

In [ ]:
y

### --------------------------------------------------------Processing Validation Set-------------------------------------------------------

In [ ]:
labels_val = []

for i in range(len(answerList_val)):
    labels_val.append( mapping[ answerList_val[i] ] )

In [ ]:
len(labels_val)

In [ ]:
scores_val = []

for i in tqdm(range(len(answerList_val))):
    
    s = [0] * (numOfClasses+1)
    s[ mapping[ answerList_val[i]] ] = 1
    
    scores_val.append(s)

In [ ]:
len(scores_val)

In [ ]:
imgPathList_val = []
filepath = '/home/aritra/cric/images/img/'

for i in tqdm(range(len(imgList_val))):
    
    imgName = str(imgList_val[i]) + '.jpg'
    concatedPath = os.path.join(filepath,imgName)
    
    imgPathList_val.append(concatedPath)

In [ ]:
imgPathList_val[0:5]

In [ ]:
# creating HF dataset to map images fast of Val_set

listToDictionary = {'questions':questionList_val, 'labels':labels_val, 'scores':scores_val, 'images':imgPathList_val}
modified_val_set = Dataset.from_dict(listToDictionary)

In [ ]:
# mapping each filepath of Val Set to images in the directory

modified_val_set = modified_val_set.cast_column("images", datasets.Image())

### -------------------------------------------Extracting Test Set-------------------------------------------------


In [ ]:
questionList_test = []
answerList_test = []
imgList_test = []

In [ ]:
indexToExcludeTest = []

with open('error_testSet1.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExcludeTest.append(number)
        
with open('errorTestSet2.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExcludeTest.append(number)

In [ ]:
len(indexToExcludeTest)

In [ ]:
for i in tqdm(range(len(test_json))):
    
    if i in indexToExcludeTest:
        continue
        
    pointer = test_json[i]
    
    questionList_test.append(pointer['question'])
    answerList_test.append(pointer['answer'])
    imgList_test.append(pointer['image_id'])

86003 -> 71863

### -------------------------------------- Processing Test Set ----------------------------------------------------------------------------

In [ ]:
# check if all the uniques answers are present in the mapping

y,n = 0,0
store = []
for i in range(len(answerList_test)):
    
    word = answerList_test[i]
    
    if word in mapping:
        y += 1
    else:
        n+=1
        store.append(i)

In [ ]:
y

In [ ]:
labels_test = []

for i in range(len(answerList_test)):
    labels_test.append( mapping[ answerList_test[i] ] )

In [ ]:
len(labels_test)

In [ ]:
scores_test = []

for i in tqdm(range(len(answerList_test))):
    
    s = [0] * (numOfClasses+1)
    s[ mapping[ answerList_test[i]] ] = 1
    
    scores_test.append(s)

In [ ]:
len(scores_test)

In [ ]:
imgPathList_test = []
filepath = '/home/aritra/cric/images/img/'

for i in tqdm(range(len(imgList_test))):
    
    imgName = str(imgList_test[i]) + '.jpg'
    concatedPath = os.path.join(filepath,imgName)
    
    imgPathList_test.append(concatedPath)

In [ ]:
len(imgPathList_test)

In [ ]:
imgPathList_test[0:5]

In [ ]:
# creating HF dataset to map images fast of test_set

listToDictionary = {'questions':questionList_test, 'labels':labels_test, 'scores':scores_test, 'images':imgPathList_test}
modified_test_set = Dataset.from_dict(listToDictionary)

In [ ]:
# mapping each filepath of test Set to images in the directory

modified_test_set = modified_test_set.cast_column("images", datasets.Image())

### -------------------------------End of Processing----------------------------------------------------------------------------

In [ ]:
from transformers import ViltProcessor, ViltForQuestionAnswering

In [ ]:
from transformers import ViltConfig
config = ViltConfig.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")

In [ ]:
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm", id2label = reverse_mapping, label2id = mapping).to(device)

In [ ]:
from torch.utils.data import DataLoader
from datasets import Dataset
import numpy as np

In [ ]:
class cric_dataset(Dataset):
    
    def __init__(self, dataset, processor):
        self.processor = processor
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self,idx):
        
        #print(idx)
        item = self.dataset[idx]

        #print(item)
        
        encodings = self.processor(images = item["images"], text = item["questions"], padding="max_length", truncation=True, return_tensors = "pt")
        encodings = {k:v.squeeze() for k,v in encodings.items()}
                                
        encodings['labels'] = torch.tensor(item['scores'], dtype = torch.float32)
        
        return encodings

In [ ]:
train_dataset_object = cric_dataset(modified_train_set, processor)

In [ ]:
val_dataset_object = cric_dataset(modified_val_set, processor)

In [ ]:
test_dataset_object = cric_dataset(modified_test_set, processor)

In [ ]:
def collate_fn(batch):
  
    input_ids = [item['input_ids'] for item in batch]
    pixel_values = [item['pixel_values'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    token_type_ids = [item['token_type_ids'] for item in batch]
    labels = [item['labels'] for item in batch]
        
    # create padded pixel values and corresponding pixel mask
    
    encoding = processor.image_processor.pad(pixel_values, return_tensors="pt")

    # create new batch
    
    batch = {}
    
    batch['input_ids'] = torch.stack(input_ids)
    batch['attention_mask'] = torch.stack(attention_mask)
    batch['token_type_ids'] = torch.stack(token_type_ids)
    batch['pixel_values'] = encoding['pixel_values']
    batch['pixel_mask'] = encoding['pixel_mask']
    batch['labels'] = torch.stack(labels, dim = 0 )

    return batch


In [ ]:
train_dataloader = DataLoader(train_dataset_object, collate_fn = collate_fn, shuffle = True, batch_size = 16)

In [ ]:
batch = next(iter(train_dataloader))

In [ ]:
for k,v in batch.items():
    print(k, v.shape)
    print()
    
#print(batch.keys())

In [ ]:
tot_number_of_steps = len(train_dataloader)
tot_number_of_steps

In [ ]:
# for fp16 precision

from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler

scaler = GradScaler()

In [ ]:
# No Visualisation for this Test Notebook

#from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter()

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 8e-5)

model.train()

for epoch in tqdm(range(10)):  

    print(f"Epoch: {epoch}")

    for idx, batch in enumerate(train_dataloader):

        batch = {k:v.to(device) for k,v in batch.items()}

        optimizer.zero_grad()
        
        with torch.autocast(device_type='cuda', dtype=torch.float16):

            outputs = model(**batch)
            loss = outputs.loss

        print(idx,"-> Loss:", loss.item())
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)

        scaler.update()
        
        # Plots in tensorboard
    
        if (idx != 0 ) and (idx % 100 == 0):
            
            model.eval()
            
            acc_score_test = calculateAccuracyTest()
            acc_score_val, validationLoss = calculateAccuracyVal()
            
            print(f'\nValidation Accuracy: {acc_score_val}, Test Accuracy: {acc_score_test} \n')
            
            #writer.add_scalar('Training Loss', loss.item(), epoch * tot_number_of_steps + idx)
            #writer.add_scalar('Validation Loss', validationLoss, epoch * tot_number_of_steps + idx)

            #writer.add_scalar('Accuracy Score On Val Set', acc_score_val, epoch * tot_number_of_steps + idx)
            #writer.add_scalar('Accuracy Score On Test Set', acc_score_test, epoch * tot_number_of_steps + idx)
            
            model.train()
            
    # Save model checkpoint
    
    save_path = os.path.join('./model_chkpts/test/', 'vilt_mlm_final_e' + str(epoch+1) + '_cric_trained')
    model.save_pretrained(save_path)
    print("Model Saved At: ", epoch)
    
#writer.close()



In [ ]:
model.eval()

In [ ]:
index = 1

In [ ]:
example = val_dataset_object[index]
print(example.keys())

In [ ]:
delLab = example.pop('labels')

In [ ]:
processor.decode(example['input_ids'])

In [ ]:
# add batch dimension + move to GPU
example = {k: v.unsqueeze(0).to(device) for k,v in example.items()}

# forward pass
outputs = model(**example)

In [ ]:
logits = outputs.logits
logits.shape

In [ ]:
logits

In [ ]:
print(torch.argmax(logits).item())
reverse_mapping[logits.argmax(-1).item()]

In [ ]:
answerList[index]

In [ ]:
predicted_classes = torch.sigmoid(logits)
probs, classes = torch.topk(predicted_classes, 5)

for prob, class_idx in zip(probs.squeeze().tolist(), classes.squeeze().tolist()):
  print(prob, model.config.id2label[class_idx])

In [ ]:
i = Image.open(imgPathList[index])
i.thumbnail((300,300))
i

In [ ]:
# This function returns the Validation Loss and accuracy on the Validation Set

def calculateAccuracyVal():
    
    matchScore, loopCounter = 0,0
    
    for index in range(0,100):
        
        loopCounter += 1
        
        val_example = val_dataset_object[index]
        val_example = {k: v.unsqueeze(0).to(device) for k,v in val_example.items()}
        val_outputs = model(**val_example)
        
        validationLoss = val_outputs.loss

        val_logits = val_outputs.logits
        val_predicted_classes = torch.sigmoid(val_logits)
        val_ans = reverse_mapping[torch.argmax(val_predicted_classes).item()]
        
        #print(f'T: {answerList_val[index]} <-> P: {val_ans}' )

        # accuracy score
        
        if answerList_val[index] == val_ans:
            matchScore += 1
                
    #print(matchScore, loopCounter)
    accuracyVal = (matchScore/loopCounter)*100
    return ( accuracyVal,validationLoss.item() )

In [ ]:
# This function returns accuracy on the Test Set
# However, accuracy needs to be found out on the whole Test Set using the saved model chkpts

def calculateAccuracyTest():
    
    matchScore, loopCounter = 0,0
    model.eval()
    for index in range(0,200):
        
        loopCounter += 1
        
        test_example = test_dataset_object[index]
        test_example = {k: v.unsqueeze(0).to(device) for k,v in test_example.items()}
        test_outputs = model(**test_example)

        test_logits = test_outputs.logits
        test_predicted_classes = torch.sigmoid(test_logits)
        test_ans = reverse_mapping[torch.argmax(test_predicted_classes).item()]
        
        # print(f'T: {answerList_val[index]} <-> P: {test_ans}' )

        # accuracy score
        
        if answerList_test[index] == test_ans:
            matchScore += 1
                
    #print(matchScore, loopCounter)
    return ((matchScore/loopCounter)*100)

In [ ]:
calculateAccuracyTest()